In [ ]:
import numpy as np
import os
import pandas
import json
from pathlib import Path
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
import itertools

In [ ]:
rows = ["experiment_1", "experiment_2", "experiment_3"]
cols = ["experiment_name", "method_name", "inference_time (something proportional to time)"
       , "ground_truth_traj", "eval_traj", "l2_eval_error", "training_loss", "model_parameters"]

In [ ]:
aggregate_data = pandas.read_pickle("../aggregate_data.pkl")
ground_truth_data = pandas.read_pickle("../ground_truth_data.pkl")
inferred_data = pandas.read_pickle("../inferred_data.pkl")
aggregate_data.columns
#inferred_data.columns

In [ ]:
pandas\
.merge(aggregate_data, inferred_data, on="experiment_name")\
.groupby(["method_name", "system_name", "num_train_trajectories"])\
.agg({"relerr_l2" : "mean"})

In [ ]:
fig, ax = plt.subplots(2, 2)
fig.set_figwidth(30)
fig.set_figheight(20)

df = pandas\
.merge(aggregate_data, inferred_data, on="experiment_name")\
.groupby(["method_name", "num_train_trajectories", "timestep_number"])\
.agg({"relerr_l2" : "mean"}).add_prefix("mean_").reset_index()

df.loc[df["method_name"] == "mlp"]\
.pivot(index="timestep_number", columns=["method_name", "num_train_trajectories"], values="mean_relerr_l2")\
.plot(ax=ax[0,0], logy=True)

df.loc[df["method_name"] == "hnn"]\
.pivot(index="timestep_number", columns=["method_name", "num_train_trajectories"], values="mean_relerr_l2")\
.plot(ax=ax[1,0], logy=True)

df.loc[df["method_name"] == "srnn"]\
.pivot(index="timestep_number", columns=["method_name", "num_train_trajectories"], values="mean_relerr_l2")\
.plot(ax=ax[0,1], logy=True)

df.loc[df["method_name"] == "knn-regressor"]\
.pivot(index="timestep_number", columns=["method_name", "num_train_trajectories"], values="mean_relerr_l2")\
.plot(ax=ax[1,1], logy=True)

def style_update(ax):
    ax.grid()
    ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    ax.legend(loc="lower right")
    

[style_update(ax) for ax in itertools.chain(*ax)]


In [ ]:
fig, ax = plt.subplots(1, 2)
fig.set_figwidth(30)
fig.set_figheight(10)

df = pandas\
.merge(aggregate_data, inferred_data, on="experiment_name")\
.groupby(["method_name", "system_name", "num_train_trajectories"])\
.agg({"relerr_l2" : "mean"}).add_prefix("mean_").reset_index()

df.loc[df["system_name"] == "spring"]\
.pivot(index="method_name", columns=["num_train_trajectories"], values="mean_relerr_l2")\
.plot(ax=ax[0], kind="bar", logy=True)

df.loc[df["system_name"] == "spring"]\
.pivot(index="num_train_trajectories", columns=["method_name"], values="mean_relerr_l2")\
.plot(ax=ax[1], kind="bar", logy=True)

def style_update(ax):
    ax.grid()
    ax.grid(which='minor', linestyle=':', linewidth='0.5', color='black')
    ax.legend(loc="upper left")

[style_update(ax_) for ax_ in ax]